In [1]:
import pandas as pd
import datetime as datetime
import os
# ! pip install xlsxwriter --user
from IPython.display import HTML

def df_horizontal(dfs):
    html = '<div style="display:flex">'
    for df in dfs:
        html += '<div style="margin-right: 32px">'
        html += df.to_html()
        html += '</div>'
    html += '</div>'
    display(HTML(html))
    

In [3]:
data_path=r'D:\PythonDev\MyQuantFinProject\Data'
data_file='World_Sep-Nov-23.csv'
isPy=False 
isPriceExcel=True

# Specify Period and Timefame to caculate fund performance and

In [4]:
# If you want to see July-Oct, you have to have last trading day of June
from_str='2022-08-01'  
to_str='2023-11-30' 
timeframe='M'

pct_return_col=f"pct_change_{timeframe}"
std_col="std_of_pct_Daily"
# timeframe='W-FRI'

# Input data

In [5]:
if isPy:
    print(f"Data csv file in Path: {data_path}")
    data_file=input("CSV file name : ")

    from_str=input("From_date(2020-10-01) : ") 
    to_str=input("To_date(2022-12-31): ") 
    timeframe=input("Timeframe(W-FRI,M,Q,Y): ") 
  
    print(f"Process data from {from_str} to {to_str} in {timeframe} timeframe")
    press_y=input(f"Press y=True and n=False to run analaystics : ") 
    if press_y.lower()=='y':
     ok=True
    else:
     exit()


In [6]:
report_prefix_name=data_file.split('.')[0]
data_file=os.path.join(data_path,data_file)
if os.path.exists(data_file)==False:
    raise Exception(f"not found {data_file}")
else:
    print(f"Process file {data_file}")
print(data_file)

Process file D:\PythonDev\MyQuantFinProject\Data\World_Sep-Nov-23.csv
D:\PythonDev\MyQuantFinProject\Data\World_Sep-Nov-23.csv


# Read csv file and extract and transform data as need

In [7]:
df = pd.read_csv(data_file,index_col='Date/Time',parse_dates=['Date/Time'],dayfirst=True)
df.index.set_names('date',inplace=True)
df.sort_index(inplace=True)
df=df.rename(columns={'Ticker':'symbol','close':'price'})
df=df[['symbol','price']]

#test 1
# df=df.query("symbol not in ('BTC-USD')")
# print(df.info())

df.tail(10)

,symbol,price
date,,
2023-11-24,AAXJ,65.74
2023-11-24,ACWI,97.88
2023-11-27,QQQ,389.17
2023-11-27,SPY,454.48
2023-11-27,ACWI,97.62
2023-11-27,AAXJ,65.38
2023-11-28,ACWI,97.78
2023-11-28,AAXJ,65.84
2023-11-28,QQQ,390.19


In [8]:
list_symbol=df['symbol'].unique().tolist() 

print(list_symbol)


['AAXJ', 'QQQ', 'ACWI', 'SPY']


# Basic

# Resample data and calculate Percentage of change

In [9]:
# colsROC=['symbol','date','pct_change','std_of_daily_pct','return_to_std']
colsROC=['symbol','date',pct_return_col,std_col]
df_Pct_Price=pd.DataFrame(columns=colsROC)
print(df_Pct_Price)

#https://pandas.pydata.org/docs/reference/api/pandas.core.resample.Resampler.aggregate.html
#https://stackoverflow.com/questions/47938372/pandas-dataframe-resample-aggregate-function-use-multiple-columns-with-a-customi
#https://www.alpharithms.com/aggregating-time-series-data-with-pandas-resampling-411212/
# https://sparkbyexamples.com/pandas/pandas-aggregate-functions-with-examples/
for symbol_name in list_symbol:
    print(f"===================================={symbol_name}====================================")
    dfx_symbol=df.query('symbol==@symbol_name')
    dfx_symbol['_pct_compare_prev_']=dfx_symbol['price'].pct_change()
    dfx_symbol=dfx_symbol.loc[from_str:to_str]
    #print(dfx_symbol.tail())
    
    
    dfx_px=dfx_symbol.resample(timeframe).agg({'price': 'last','_pct_compare_prev_':'std'}) 
    dfx_px=dfx_px.rename(columns={'_pct_compare_prev_':std_col})

    dfx_px[pct_return_col]=dfx_px['price'].pct_change()
    dfx_px[pct_return_col]=dfx_px[pct_return_col].round(4)

    dfx_px=dfx_px.dropna() # the first  rown 
    dfx_px=dfx_px.drop(columns=['price']) # remove price becase we are focuse ROC of Price
    
    dfx_px[pct_return_col]=dfx_px[pct_return_col]*100
    dfx_px[std_col]=dfx_px[std_col]*100
    
    # dfx_px[pct_return_col]=dfx_px[pct_return_col]/dfx_px[std_col]
    
    dfx_px=dfx_px.reset_index()
    dfx_px['symbol']=symbol_name
    dfx_px=dfx_px[colsROC]
    
#     print(f"Calculate ROC of {symbol_name}")
    print(dfx_px)
    
    df_Pct_Price=pd.concat([df_Pct_Price,dfx_px],ignore_index=True,axis=0)

Empty DataFrame
Columns: [symbol, date, pct_change_M, std_of_pct_Daily]
Index: []
====================================AAXJ====================================
  symbol       date  pct_change_M  std_of_pct_Daily
0   AAXJ 2023-09-30         -3.17          0.880001
1   AAXJ 2023-10-31         -3.29          0.976224
2   AAXJ 2023-11-30          7.21          1.015098
====================================QQQ====================================
  symbol       date  pct_change_M  std_of_pct_Daily
0    QQQ 2023-09-30         -5.22          0.889364
1    QQQ 2023-10-31         -2.07          1.189801
2    QQQ 2023-11-30         11.21          0.918072
====================================ACWI====================================
  symbol       date  pct_change_M  std_of_pct_Daily
0   ACWI 2023-09-30         -4.28          0.658931
1   ACWI 2023-10-31         -2.54          0.857609
2   ACWI 2023-11-30          8.62          0.748724
====================================SPY=========================

C:\Users\pongthsa\AppData\Local\Temp\ipykernel_10256\317653757.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfx_symbol['_pct_compare_prev_']=dfx_symbol['price'].pct_change()
C:\Users\pongthsa\AppData\Local\Temp\ipykernel_10256\317653757.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfx_symbol['_pct_compare_prev_']=dfx_symbol['price'].pct_change()
C:\Users\pongthsa\AppData\Local\Temp\ipykernel_10256\317653757.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

In [10]:
print("===============List All================================")
print(df_Pct_Price)

===============List All================================
   symbol       date  pct_change_M  std_of_pct_Daily
0    AAXJ 2023-09-30         -3.17          0.880001
1    AAXJ 2023-10-31         -3.29          0.976224
2    AAXJ 2023-11-30          7.21          1.015098
3     QQQ 2023-09-30         -5.22          0.889364
4     QQQ 2023-10-31         -2.07          1.189801
5     QQQ 2023-11-30         11.21          0.918072
6    ACWI 2023-09-30         -4.28          0.658931
7    ACWI 2023-10-31         -2.54          0.857609
8    ACWI 2023-11-30          8.62          0.748724
9     SPY 2023-09-30         -5.08          0.716959
10    SPY 2023-10-31         -2.17          0.880649
11    SPY 2023-11-30          8.78          0.734806


# Total Period : Summarize and Combinำ return and std

In [12]:
print(f"==================Sum-Return and Avg-Std based on {timeframe}===========================")
print("Sum percentage of fund return")
dfSum_Pct_Price=df_Pct_Price.groupby(['symbol'])[[pct_return_col]].sum()
dfSum_Pct_Price.reset_index(drop=False,inplace=True)
dfSum_Pct_Price[pct_return_col]=dfSum_Pct_Price[pct_return_col].round(1)


print("==============================================================")
print("Average Std of fund ")
dfAvg_Std_Price=df_Pct_Price.groupby(['symbol'])[[std_col]].mean()
dfAvg_Std_Price.reset_index(drop=False,inplace=True)
dfAvg_Std_Price[std_col]=dfAvg_Std_Price[std_col].round(2)

dfValueSummary=pd.merge(left=dfSum_Pct_Price,right=dfAvg_Std_Price,on='symbol')
dfValueSummary.fillna(value=0,inplace=True)
dfValueSummary.set_index('symbol',inplace=True)
dfValueSummary

==================Sum-Return and Avg-Std based on M===========================
Sum percentage of fund return
Average Std of fund 


,pct_change_M,std_of_pct_Daily
symbol,,
AAXJ,0.8,0.96
ACWI,1.8,0.76
QQQ,3.9,1.00
SPY,1.5,0.78


# Timeframe Period 

# Pivot Percent Return & Std and theire Rank To Dataframe

In [13]:
print("==============================================================")
print(f"Pivot Percent Return and Rank {timeframe} To Dataframe")
dfPct=df_Pct_Price.pivot(index='date', columns='symbol', values=pct_return_col)
dfRank=dfPct[dfPct.columns.tolist()].rank(axis=1)
# print(dfPct.info())

Pivot Percent Return and Rank M To Dataframe


In [14]:
print("==============================================================")
print(f"Pivot Daily STD and Rank in {timeframe} and Rank To Dataframe")
dfStd=df_Pct_Price.pivot(index='date', columns='symbol', values=std_col)
dfRankStd=dfStd[dfStd.columns.tolist()].rank(axis=1,ascending=False)


Pivot Daily STD and Rank in M and Rank To Dataframe


In [15]:
df_horizontal([dfRank,dfPct])
df_horizontal([dfRankStd,dfStd])

symbol,AAXJ,ACWI,QQQ,SPY
date,,,,
2023-09-30,4.0,3.0,1.0,2.0
2023-10-31,1.0,2.0,4.0,3.0
2023-11-30,1.0,2.0,4.0,3.0
symbol,AAXJ,ACWI,QQQ,SPY
date,,,,
2023-09-30,-3.17,-4.28,-5.22,-5.08
2023-10-31,-3.29,-2.54,-2.07,-2.17
2023-11-30,7.21,8.62,11.21,8.78


symbol,AAXJ,ACWI,QQQ,SPY
date,,,,
2023-09-30,2.0,4.0,1.0,3.0
2023-10-31,2.0,4.0,1.0,3.0
2023-11-30,1.0,3.0,2.0,4.0
symbol,AAXJ,ACWI,QQQ,SPY
date,,,,
2023-09-30,0.880001,0.658931,0.889364,0.716959
2023-10-31,0.976224,0.857609,1.189801,0.880649
2023-11-30,1.015098,0.748724,0.918072,0.734806


# Combine Return and Std Rank as Score

In [16]:
print("==============================================================")
print("Combine Both Pct Return and Rank")
print("Return :higher total score , greater performance")
print("Std : higher total score , worse perfromance")
dfRankSummary=pd.concat([
                     dfRank.sum().to_frame(name='sum_return_rank'),
                     dfRankStd.sum().to_frame(name='sum_std_rank'),  
                    ],axis=1)
dfRankSummary.fillna(value=0,inplace=True)
dfRankSummary

Combine Both Pct Return and Rank
Return :higher total score , greater performance
Std : higher total score , worse perfromance


,sum_return_rank,sum_std_rank
symbol,,
AAXJ,6.0,5.0
ACWI,7.0,11.0
QQQ,9.0,4.0
SPY,8.0,10.0


# Convert value score   and rank score to final rank

In [17]:
dfValueSummary['value_return_rank']=dfValueSummary[pct_return_col].rank()
dfValueSummary['value_std_rank']=dfValueSummary[std_col].rank(ascending=False)

In [18]:
# real score
dfRankSummary['return_rank']=dfRankSummary['sum_return_rank'].rank()
dfRankSummary['std_rank']=dfRankSummary['sum_std_rank'].rank()



In [19]:
df_horizontal([dfValueSummary, dfRankSummary])

,pct_change_M,std_of_pct_Daily,value_return_rank,value_std_rank
symbol,,,,
AAXJ,0.8,0.96,1.0,2.0
ACWI,1.8,0.76,3.0,4.0
QQQ,3.9,1.00,4.0,1.0
SPY,1.5,0.78,2.0,3.0
,sum_return_rank,sum_std_rank,return_rank,std_rank
symbol,,,,
AAXJ,6.0,5.0,1.0,2.0
ACWI,7.0,11.0,2.0,4.0
QQQ,9.0,4.0,4.0,1.0


# Summarize Total Score 

In [23]:
dfFinalScore= pd.concat([ dfValueSummary[['value_return_rank','value_std_rank']],dfRankSummary[['return_rank','std_rank']]],axis=1)
dfFinalScore['TotalScore']=dfFinalScore.sum(axis=1)
dfFinalScore=dfFinalScore.sort_values(by='TotalScore',ascending=False)

# dfFinalScore['pctChangeScore']=dfFinalScore['TotalScore'].pct_change()*100
# dfFinalScore['pctChangeScore']=dfFinalScore['pctChangeScore'].round(0)
# dfFinalScore.fillna(value=0,inplace=True)

# dfFinalScore=dfFinalScore[['TotalScore','pctChangeScore','value_return_rank','value_std_rank','return_rank','std_rank']]
dfFinalScore=dfFinalScore[['TotalScore','value_return_rank','value_std_rank','return_rank','std_rank']]
dfFinalScore

,TotalScore,value_return_rank,value_std_rank,return_rank,std_rank
symbol,,,,,
ACWI,13.0,3.0,4.0,2.0,4.0
SPY,11.0,2.0,3.0,3.0,3.0
QQQ,10.0,4.0,1.0,4.0,1.0
AAXJ,6.0,1.0,2.0,1.0,2.0


In [24]:
if isPriceExcel:
    print("Export outcoome as Excel")
    with pd.ExcelWriter(f"{report_prefix_name}_{timeframe}_{from_str}To{to_str}.xlsx") as writer:
        df_Pct_Price.to_excel(writer, sheet_name=f"{timeframe}")
        
        dfPct.to_excel(writer, sheet_name=f"{timeframe}-Return")
        dfRankStd.to_excel(writer, sheet_name=f"{timeframe}-RankStd")
        
        dfStd.to_excel(writer, sheet_name=f"{timeframe}-Std")
        dfRank.to_excel(writer, sheet_name=f"{timeframe}-RankReturn")
        
        
        dfValueSummary.to_excel(writer, sheet_name="Value-Summary")
        dfRankSummary.to_excel(writer, sheet_name="Rank-Summary")

        dfFinalScore.to_excel(writer, sheet_name="FinalScore")

Export outcoome as Excel
